In [4]:
import os
import re
import json

In [5]:
def countPipes(line):
    return line.count(' :') + bool(re.search(r"\+-", line))



def complete_and_save(debugLines):
    branches = 1 # we assusme that there is at least one branch
    id = 1
    result = []
    split_ids = []
    
    for line in reversed(debugLines):
        if (not "DeserializeToObject" in line and not "SerializeFromObject" in line):
            pipesInLine = countPipes(line)
            if (pipesInLine == branches) or (pipesInLine == 0):
                #data source
                if (id-1 == 0):
                    result.append({'id': id, 'type': 'Data Source', 'operator': re.search(r"[A-Z]\S+", line).group()})
                    id += 1
                elif (pipesInLine != 0):
                    result.append({'id': id, 'type': 'operator', 'operator': re.search(r"[A-Z]\S+", line).group(), 'predecessors': [{'id': id-1}]})
                    id += 1
                #data sink
                else:
                    result.append({'id': id, 'type': 'operator', 'operator': re.search(r"[A-Z]\S+", line).group(), 'predecessors': [{'id': id-1}]})
                    id += 1
                    result.append({'id': id, 'type': 'sink', 'operator': 'Data Sink', 'predecessors': [{'id': id-1}]})
                    id += 1
            else:
                #starting new branch
                if pipesInLine > branches:
                    split_ids.append(id-1)
                    result.append({'id': id, 'type': 'Data Source', 'operator': re.search(r"[A-Z]\S+", line).group()})
                    id += 1
                    branches += 1

                    
                #merging branches, but check for :- case
                else:
                    if bool(re.search(r":-", line)):
                        result.append({'id': id, 'type': 'operator', 'operator': re.search(r"[A-Z]\S+", line).group(), 'predecessors': [{'id': id-1}]})
                        id += 1
                    else:
                        result.append({'id': id, 'type': 'operator', 'operator': re.search(r"[A-Z]\S+", line).group(), 'predecessors': [{'id': id-1}, {'id': split_ids.pop()}]})
                        id += 1
                        branches -= 1
                        
            
        
        
    return {"executionPlan" : {"nodes" : result}}

In [6]:
directory = 'logicalPlans'
output_dir = 'unified_input'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
    #and (filename == "Q01optimizedPlan.txt" or filename == "Q03optimizedPlan.txt" or filename == "Q11optimizedPlan.txt"
    #                          or filename == "Q13optimizedPlan.txt" or filename == "Q17optimizedPlan.txt" or filename == "Q21optimizedPlan.txt"):
        debugString = open(f, "r")
        debugLines = debugString.readlines()
        
        
        unified_json = complete_and_save(debugLines)
        #print(*unified_json, sep="\n")
        
        with open(os.path.join(output_dir, 'TPC_H_' + filename.split('.')[0] + 'Unified.json'), 'w', encoding='utf-8') as nf:
            json.dump(unified_json, nf, ensure_ascii=False, indent=2)
        